# Повний цикл NLP-проєкту

## I. Перевірка фактів на достовірність

У межах цієї задачі ви побудуєте систему видобування фактів на правилах, а також інструменти для оцінювання якості роботи цієї системи.

### 1. Домен

Виберіть домен, для якого можна побудувати невелику базу даних на основі [DBPedia](https://dbpedia.org/sparql). База даних повинна містити хоча б три колонки з даними.

```
    SELECT ?cyclist ?name ?ridertype ?wiki
        (GROUP_CONCAT(DISTINCT ?place1; SEPARATOR=", ") AS ?places1)
        (GROUP_CONCAT(DISTINCT ?place2; SEPARATOR=", ") AS ?places2)
        (GROUP_CONCAT(DISTINCT ?place3; SEPARATOR=", ") AS ?places3)
    WHERE {
        ?cyclist dct:subject dbc:Ukrainian_male_cyclists;
        dct:subject dbc:Tour_de_France_cyclists;
        rdfs:label ?name;
        foaf:isPrimaryTopicOf ?wiki.
        OPTIONAL {?cyclist dbp:ridertype ?ridertype}.
        OPTIONAL {?place1 dbp:first ?cyclist}.
        OPTIONAL {?place2 dbp:second ?cyclist}.
        OPTIONAL {?place3 dbp:third ?cyclist}.
        FILTER (langmatches(lang(?name),  "EN"))
    }
    GROUP BY ?cyclist ?name ?ridertype ?wiki
    ORDER BY ?cyclist
```

Переверіри результати можна [тут](http://dbpedia.org/snorql/?query=select+%3Fcyclist+%3Fname+%3Fridertype++%3Fwiki%0D%0A+%28GROUP_CONCAT%28DISTINCT+%3Fplace1%3B+SEPARATOR%3D%22%2C+%22%29+AS+%3Fplaces1%29%0D%0A+%28GROUP_CONCAT%28DISTINCT+%3Fplace2%3B+SEPARATOR%3D%22%2C+%22%29+AS+%3Fplaces2%29%0D%0A+%28GROUP_CONCAT%28DISTINCT+%3Fplace3%3B+SEPARATOR%3D%22%2C+%22%29+AS+%3Fplaces3%29%0D%0Awhere+%7B%0D%0A++++%3Fcyclist+dct%3Asubject+dbc%3AUkrainian_male_cyclists%3B%0D%0A++++dct%3Asubject+dbc%3ATour_de_France_cyclists%3B%0D%0A++++rdfs%3Alabel+%3Fname%3B%0D%0A++++foaf%3AisPrimaryTopicOf+%3Fwiki.%0D%0A++++OPTIONAL+%7B%3Fcyclist+dbp%3Aridertype+%3Fridertype%7D.%0D%0A++++OPTIONAL+%7B%3Fplace1+dbp%3Afirst+%3Fcyclist%7D.%0D%0A++++OPTIONAL+%7B%3Fplace2+dbp%3Asecond+%3Fcyclist%7D.%0D%0A++++OPTIONAL+%7B%3Fplace3+dbp%3Athird+%3Fcyclist%7D.%0D%0A++++FILTER+%28langmatches%28lang%28%3Fname%29%2C++%22EN%22%29%29%0D%0A%7D%0D%0AGROUP+BY+%3Fcyclist+%3Fname+%3Fridertype+%3Fwiki%0D%0AORDER+BY+%3Fcyclist%0D%0A)


In [248]:
from collections import defaultdict
import json
import pandas as pd
import urllib.parse

In [214]:
def strip_places(places):
    places2 = [
        (
            p.replace('http://dbpedia.org/resource/', '')
            .replace('_', ' ')
            .replace('–', '-') # unicode char
        ) for p in places.split(', ')
    ]
    return [p for p in places2 if p]

In [215]:
def read_raw_data(filename):
    with open(filename, 'r') as f:
        raw_data = json.load(f)

    data = defaultdict(dict)
    for cyclist in raw_data['results']['bindings']:
        cyclist_id = cyclist['cyclist']['value'].replace('http://dbpedia.org/resource/', '')
        data[cyclist_id]['name'] = cyclist['name']['value']
        data[cyclist_id]['rider_type'] = cyclist.get('ridertype', {}).get('value')
        data[cyclist_id]['wiki_url'] = cyclist['wiki']['value']
        data[cyclist_id]['places1'] = strip_places(cyclist['places1']['value'])
        data[cyclist_id]['places2'] = strip_places(cyclist['places2']['value'])
        data[cyclist_id]['places3'] = strip_places(cyclist['places3']['value'])

    return data

In [217]:
dbpedia_cyclists_df = pd.DataFrame(read_raw_data('raw_cyclists_data.json')).T
dbpedia_cyclists_df

,name,rider_type,wiki_url,places1,places2,places3
Andrei_Tchmil,Andrei Tchmil,Classics specialist,http://en.wikipedia.org/wiki/Andrei_Tchmil,"[1994 Paris-Roubaix, 1999 Milan-San Remo, 2000...",[1995 Paris-Roubaix],"[1994 Tour of Flanders, 1995 Tour of Flanders,..."
Andriy_Hrivko,Andriy Hrivko,Puncheur/Time-trialist,http://en.wikipedia.org/wiki/Andriy_Hrivko,[],[],[]
Denys_Kostyuk,Denys Kostyuk,None,http://en.wikipedia.org/wiki/Denys_Kostyuk,[],[],[]
Serguei_Outschakov,Serguei Outschakov,None,http://en.wikipedia.org/wiki/Serguei_Outschakov,[],[],[]
Serhiy_Honchar,Serhiy Honchar,Time trialist,http://en.wikipedia.org/wiki/Serhiy_Honchar,[],[2004 Giro d'Italia],[1998 UCI Road World Championships - Men's tim...
Volodymir_Hustov,Volodymir Hustov,Climber,http://en.wikipedia.org/wiki/Volodymir_Hustov,[],[],[]
Yaroslav_Popovych,Yaroslav Popovych,All-rounder,http://en.wikipedia.org/wiki/Yaroslav_Popovych,[2005 Volta a Catalunya],[],"[2003 Giro d'Italia, 2006 Tour de Georgia, 200..."
Yuriy_Krivtsov,Yuriy Krivtsov,None,http://en.wikipedia.org/wiki/Yuriy_Krivtsov,[],[],[]


In [225]:
tuple(dbpedia_cyclists_df.loc[:, 'wiki_url'].to_dict().values())

('http://en.wikipedia.org/wiki/Andrei_Tchmil',
 'http://en.wikipedia.org/wiki/Andriy_Hrivko',
 'http://en.wikipedia.org/wiki/Denys_Kostyuk',
 'http://en.wikipedia.org/wiki/Serguei_Outschakov',
 'http://en.wikipedia.org/wiki/Serhiy_Honchar',
 'http://en.wikipedia.org/wiki/Volodymir_Hustov',
 'http://en.wikipedia.org/wiki/Yaroslav_Popovych',
 'http://en.wikipedia.org/wiki/Yuriy_Krivtsov')

## Трохи висновків по dbpedia

Я намагався отримати дані про команди, але схоже dbpedia зберігає тільки нинішню команду спортсмена, а українські спортсмени припинили змагатися, тож цієї інформації нема. З иншого боку згадка про команди на dbpedia все ж є, але вона зібрана абсолютно безсистемно, як якісь теги впереміш з иншими тегами. Тож тут dbpedia вже підвела

### 2. Видобування фактів

2.1. Напишіть програму, яка шукає статті у Вікіпедії про сутності, що належать до вашого домена, та витягає тексти цих статей.

Отак виглядає мій павук, що витягає дані з wikipedia

```
# -*- coding: utf-8 -*-
import re
import unicodedata
import scrapy
from bs4 import BeautifulSoup


class WikipediaSpider(scrapy.Spider):
    name = 'wikipedia'
    allowed_domains = ['en.wikipedia.org']
    start_urls = [
        'http://en.wikipedia.org/wiki/Andrei_Tchmil',
        'http://en.wikipedia.org/wiki/Andriy_Hrivko',
        'http://en.wikipedia.org/wiki/Denys_Kostyuk',
        'http://en.wikipedia.org/wiki/Serguei_Outschakov',
        'http://en.wikipedia.org/wiki/Serhiy_Honchar',
        'http://en.wikipedia.org/wiki/Volodymir_Hustov',
        'http://en.wikipedia.org/wiki/Yaroslav_Popovych',
        'http://en.wikipedia.org/wiki/Yuriy_Krivtsov'
    ]

    def _clean(self, value):
        #value = value.replace("\n", "").replace("\xa0", "")
        value = value.replace('–', '-')
        value = re.sub(r' , ', ', ', value)
        value = re.sub(r' \( ', ' (', value)
        value = re.sub(r' \) ', ') ', value)
        value = re.sub(r' \)', ') ', value)
        value = re.sub(r'\[\d.*\]', ' ', value)
        value = re.sub(r' +', ' ', value)
        return value.strip()

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        rows = []
        nodes = response.css("div#mw-content-text>div.mw-parser-output>*").getall()

        for html in nodes:
            soup = BeautifulSoup(html.replace('<', ' <'), 'lxml')
            rows.append(self._clean(soup.text))

        yield {
            'url': response.url,
            'title': response.css('#firstHeading::text').extract_first(),
            'rows': rows
        }
```

2.2. Напишіть програму, яка опрацьовує текст статті (саме сирий текст, а не таблички, якщо такі є) та витягає з нього інформацію про ваш домен. Цю інформацію ви будете порівнювати зі сформованою базою даних.

In [250]:
with open('/Users/dmytrobudashnyi/usr/projects/nlp_course/world_scrapah/world_scrapah/cyclists.json') as f:
    cyclists_raw_data = json.load(f)

In [251]:
import re

win_pattern = re.compile(r"^1st\s+([\w\-\s]+)")
place2_pattern = re.compile(r"^2nd\s+([\w\-\s]+)")
place3_pattern = re.compile(r"^3rd\s+([\w\-\s]+)")
year_pattern = re.compile(r"^(\d{4})$")
stage_pattern = re.compile(r".([Ss]tages?\s+\d+).*")
rider_type_pattern = re.compile(r".Rider\s+type\s([\w\-\\]+)\s+.*")

def parse_cyclist(data):
    result = {
        'name': data['title'],
        'rider_type': None,
        'wiki_url': data['url'],
        'places1': [],
        'places2': [],
        'places3': [],
    }
    rows = []
    for row in data['rows']:
        rows.extend([r.strip() for r in row.split("\n") if r.strip()])

    current_year = None
    for row in rows:
        #print('|{}|'.format(row))
        year_data = year_pattern.findall(row)
        if year_data:
            current_year = year_data[0]
        # 1st place
        win_data = win_pattern.findall(row)
        if win_data and not stage_pattern.search(row):
            race =  win_data[0].replace('Overall', '').strip()
            result['places1'].append('{year} {race}'.format(year=current_year, race=race))
            continue
        # 2nd place
        place2_data = place2_pattern.findall(row)
        if place2_data and not stage_pattern.search(row):
            race =  place2_data[0].replace('Overall', '').strip()
            result['places2'].append('{year} {race}'.format(year=current_year, race=race))
            continue
        # 3rd place
        place3_data = place3_pattern.findall(row)
        if place3_data and not stage_pattern.search(row):
            race =  place3_data[0].replace('Overall', '').strip()
            result['places3'].append('{year} {race}'.format(year=current_year, race=race))
            continue
        # rider type
        rider_type_data = rider_type_pattern.findall(row)
        #print('|{}|'.format(row))
        if rider_type_data:
            result['rider_type'] = rider_type_data[0]
        

    return result

cyclists_data = {}

for cyclist in cyclists_raw_data:
    cyclist_id = cyclist['url'].replace('https://en.wikipedia.org/wiki/', '')
    cyclists_data[urllib.parse.unquote(cyclist_id)] = parse_cyclist(cyclist)

wikipedia_cyclists_df = pd.DataFrame(cyclists_data).T
wikipedia_cyclists_df

,name,rider_type,wiki_url,places1,places2,places3
Andrei_Tchmil,Andrei Tchmil,Classics,https://en.wikipedia.org/wiki/Andrei_Tchmil,"[1991 Road race, 1991 Grand Prix Pino Cerami, ...","[1989 GP Industria, 1990 Grand Prix Pino Ceram...","[1989 Giro del Veneto, 1991 Coppa Bernocchi, 1..."
Yuriy_Krivtsov,Yuriy Krivtsov,None,https://en.wikipedia.org/wiki/Yuriy_Krivtsov,"[2002 Prix des Blés d, 2004 National Time Tria...","[2005 Duo Normand, 2006 National Time Trial Ch...",[2009 Chrono des Nations]
Andriy_Hrivko,Andriy Hrivko,Puncheur,https://en.wikipedia.org/wiki/Andriy_Hrivko,"[2005 Time trial, 2006 Time trial, 2008 Time t...","[2005 Firenze-Pistoia, 2008 Intaka Tech Worlds...","[2006 Critérium International, 2006 GP Miguel ..."
Denys_Kostyuk,Denys Kostyuk,None,https://en.wikipedia.org/wiki/Denys_Kostyuk,"[2013 National Road Race Championships, 2013 R...","[2008 National Road Race Championships, 2013 F...","[2003 Trofeo Banca Popolare di Vicenza, 2004 N..."
Serguei_Outschakov,Serguei Outschakov,None,https://en.wikipedia.org/wiki/Serguei_Outschakov,[],[],[]
Volodymir_Hustov,Volodymir Gustov,Climber,https://en.wikipedia.org/wiki/Volodymir_Hustov,[2003 Regio-Tour],[2000 National Road Race Championships],[]
Serhiy_Honchar,Serhiy Honchar,Time,https://en.wikipedia.org/wiki/Serhiy_Honchar,"[1997 Chrono des Herbiers, 1998 Chrono des Her...","[1997 World Time Trial Championship, 2000 Chro...","[1998 World Time Trial Championship, 2001 Chro..."
Yaroslav_Popovych,Yaroslav Popovych,All-rounder,https://en.wikipedia.org/wiki/Yaroslav_Popovych,"[2000 Tour de Nouvelle-Calédonie, 2001 Under-2...",[],"[2002 Porec Trophy III, 2003 Giro d, 2003 Tour..."


### 3. Оцінювання результатів

Розробіть метрику, яка покаже, наскільки інформація, яку ви дістали зі статей, збігається з інформацією в вашій базі даних. Скільки пропущеної інформації? Чи є часткові збіги? (Наприклад, ім'я СЕО певної компанії збігається лише частково або ім'я СЕО збігається, а роки діяльності різні.)

Додайте ваші спостереження і висновки.

In [263]:
def measure_qaulity(correct, predicted):
    qa_stats = defaultdict(int)

    for index, row in predicted.iterrows():
        correct_row = correct.loc[index , : ]
        if row['rider_type'] is None and correct_row['rider_type'] is not None:
            pass
        elif correct_row['rider_type'] is None:
            qa_stats['rider_type'] += 1
        elif row['rider_type'] == correct_row['rider_type']:
            qa_stats['rider_type'] += 1
        elif correct_row['rider_type'].startswith(row['rider_type']):
            qa_stats['rider_type'] += 0.3
        elif row['rider_type'] and not correct_row['rider_type']:
            qa_stats['rider_type'] += 1

        if not correct_row['places1']:
            qa_stats['places1'] += 1
        for race in correct_row['places1']:
            if race in row['places1']:
                qa_stats['places1'] += 1 / len(correct_row['places1'])

        if not correct_row['places2']:
            qa_stats['places2'] += 1
        for race in correct_row['places2']:
            if race in row['places2']:
                qa_stats['places2'] += 1 / len(correct_row['places2'])

        if not correct_row['places3']:
            qa_stats['places3'] += 1
        for race in correct_row['places3']:
            if race in row['places3']:
                qa_stats['places3'] += 1 / len(correct_row['places3'])

    for key in qa_stats.keys():
        qa_stats[key] /= len(correct)

    return {
        'overall': sum(qa_stats.values()) / len(qa_stats),
        'rider_type': qa_stats['rider_type'],
        'places1': qa_stats['places1'],
        'places2': qa_stats['places2'],
        'places3': qa_stats['places3']
    }

measure_qaulity(dbpedia_cyclists_df, wikipedia_cyclists_df)

{'overall': 0.8614583333333332,
 'rider_type': 0.7374999999999999,
 'places1': 1.0,
 'places2': 0.875,
 'places3': 0.8333333333333333}

## Ок, тепер те саме, тільки по великій кількості даних, які стосуються абсолютно всіх велосипедистів з TDF

In [253]:
cyclists_data = {}

for cyclist in cyclists_raw_data:
    cyclists_data[cyclist['url'].replace('https://en.wikipedia.org/wiki/', '')] = parse_cyclist(cyclist)
    
dbpedia_all_cyclists_df = pd.DataFrame(read_raw_data('raw_tdf_cyclists_data.json')).T
dbpedia_all_cyclists_df

,name,rider_type,wiki_url,places1,places2,places3
Aad_van_den_Hoek,Aad van den Hoek,None,http://en.wikipedia.org/wiki/Aad_van_den_Hoek,[],[],[]
Aart_Vierhouten,Aart Vierhouten,None,http://en.wikipedia.org/wiki/Aart_Vierhouten,[],[],[]
Abdel-Kader_Zaaf,Abdel-Kader Zaaf,None,http://en.wikipedia.org/wiki/Abdel-Kader_Zaaf,[],[],[]
Abelardo_Rondón,Abelardo Rondón,None,http://en.wikipedia.org/wiki/Abelardo_Rondón,[],[],[]
Abelardo_Ríos,Abelardo Ríos,None,http://en.wikipedia.org/wiki/Abelardo_Ríos,[],[],[]
...,...,...,...,...,...,...
Íñigo_Cuesta,Íñigo Cuesta,Domestique/Climber,http://en.wikipedia.org/wiki/Íñigo_Cuesta,[],[],[]
Óscar_Freire,Óscar Freire,Sprinter,http://en.wikipedia.org/wiki/Óscar_Freire,[1999 UCI Road World Championships - Men's roa...,[2012 E3 Harelbeke],[2000 Milan-San Remo]
Óscar_Pereiro,Óscar Pereiro,All-rounder,http://en.wikipedia.org/wiki/Óscar_Pereiro,[],[],[]
Óscar_Sevilla,Óscar Sevilla,Climbing specialist,http://en.wikipedia.org/wiki/Óscar_Sevilla,"[2008 Clásico RCN, 2013 Vuelta a Colombia, 201...","[2001 Vuelta a España, 2010 Vuelta a Colombia]",[]


In [257]:
with open('./tdf_cyclists.json') as f:
    cyclists_raw_data = json.load(f)

cyclists_tdf_data = {}

for cyclist in cyclists_raw_data:
    cyclist_id = cyclist['url'].replace('https://en.wikipedia.org/wiki/', '')
    cyclists_tdf_data[urllib.parse.unquote(cyclist_id)] = parse_cyclist(cyclist)

wikipedia_tdf_cyclists_df = pd.DataFrame(cyclists_tdf_data).T
wikipedia_tdf_cyclists_df

,name,rider_type,wiki_url,places1,places2,places3
Ad_Wijnands,Ad Wijnands,None,https://en.wikipedia.org/wiki/Ad_Wijnands,[],[],[]
Abraham_Olano,Abraham Olano,Time-trialist,https://en.wikipedia.org/wiki/Abraham_Olano,"[1992 Prueba Villafranca de Ordizia, 1994 Road...","[1995 Time trial, 1995 Vuelta a España, 1996 T...","[1992 Clasica de Almeria, 1996 Giro d, 1996 To..."
Aad_van_den_Hoek,Aad van den Hoek,None,https://en.wikipedia.org/wiki/Aad_van_den_Hoek,[1978 leg Part B Tour of Netherlands],[],[]
Abelardo_Rondón,Abelardo Rondón,None,https://en.wikipedia.org/wiki/Abelardo_Rond%C3...,[],[],[]
Abelardo_Ríos,Abelardo Ríos,None,https://en.wikipedia.org/wiki/Abelardo_R%C3%ADos,[],[],[]
...,...,...,...,...,...,...
Étienne_De_Beule,Étienne De Beule,None,https://en.wikipedia.org/wiki/%C3%89tienne_De_...,[],[],[]
Óscar_Sevilla,Óscar Sevilla,Climbing,https://en.wikipedia.org/wiki/%C3%93scar_Sevilla,"[2000 Trofeo Luis Ocaña, 2000 Memorial Manuel ...","[2000 Volta a Catalunya, 2000 Vuelta a Burgos,...","[2001 Escalada a Montjuïc, 2001 Clásica a los ..."
Óscar_Freire,Óscar Freire,Sprinter,https://en.wikipedia.org/wiki/%C3%93scar_Freire,"[1999 Road race, 2000 Points classification, 2...","[1997 Road race, 2001 Paris-Tours, 2007 Vatten...","[1998 Road race, 2000 Road race, 2000 Milan-Sa..."
Óscar_Vargas_(cyclist),Óscar Vargas (cyclist),None,https://en.wikipedia.org/wiki/%C3%93scar_Varga...,"[1986 in Subida Urkiola, 1989 in Mountains Cla...",[1990 in Colombia National Championships],"[1986 in Clasica de Sabiñanigo, 1989 in Genera..."


In [264]:
measure_qaulity(dbpedia_all_cyclists_df, wikipedia_tdf_cyclists_df)

{'overall': 0.9214897120693029,
 'rider_type': 0.8949036918138032,
 'places1': 0.9462253518824005,
 'places2': 0.9227438915641166,
 'places3': 0.9220859130168911}

## Висновки

Виявилося, що на dbpedia дуже мало даних. З wikipedia можна набагато більше стягти, це і дані про велику кількість змагань, і дані про команди з хронологією, і дані про теперішню команду, навіть якщо гонщик вже на пенсії. Тим не менш результат можна покращити, адже мої регекспи трохи не впоруються з деякими ситуаціями.

Результат на всій групі велосипедистів був кращий, тому що багато полів на dbpedia просто незаповнені.